In [ ]:
#r "nuget: Newtonsoft.Json, 13.0.1"
#r "nuget: QuikGraph, 2.3.0"
#r "/Users/jnmatthews/MGGG/GerryChain.NET/GerryChain/bin/Release/net5.0/GerryChain.dll"

Installed Packages Newtonsoft.Json, 13.0.1 QuikGraph, 2.3.0

In [ ]:
using System;
using System.Linq;
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
using GerryChain;

In [ ]:
var TallyBVAP = Scores.TallyFactory("BVAP", "BVAP");
var TallyVAP = Scores.TallyFactory("VAP", "VAP");
var TallyPOP = Scores.TallyFactory("TOTPOP", "TOTPOP");
var initPart = new Partition("../resources/al_vtds20_with_seeds.json", "CD_Seed", "TOTPOP", new string[] { "TOTPOP", "VAP", "BVAP" }, new Score[] { TallyBVAP, TallyPOP, TallyVAP });
var chain = new ReComChain(initPart, 100, 0.01, batchSize:4);

In [ ]:
var bs = chain.Select(p => ((DistrictWideScoreValue) p.Score("BVAP")).Value.Zip(((DistrictWideScoreValue)p.Score("VAP")).Value, (b, v) => b/v)).ToArray();

In [ ]:
bs.Select(v => v.OrderBy(v => v))

index,value
0,"[ 0.13258485970646555, 0.16951728221183604, 0.18607587146687765, 0.244974735891391, 0.28313392857142855, 0.3611319531617938, 0.36776413657137735 ]"
1,"[ 0.13258485970646555, 0.16951728221183604, 0.18563988946987456, 0.24531838529999608, 0.28313392857142855, 0.3611319531617938, 0.36776413657137735 ]"
2,"[ 0.13258485970646555, 0.16951728221183604, 0.18706918176775475, 0.24531838529999608, 0.28313392857142855, 0.35914404250153525, 0.36776413657137735 ]"
3,"[ 0.13258485970646555, 0.16951728221183604, 0.18706918176775475, 0.24531838529999608, 0.28313392857142855, 0.35914404250153525, 0.36776413657137735 ]"
4,"[ 0.13258485970646555, 0.16951728221183604, 0.18706918176775475, 0.24531838529999608, 0.2860378993814758, 0.35914404250153525, 0.3646277907650074 ]"
5,"[ 0.13258485970646555, 0.16951728221183604, 0.18706918176775475, 0.24531838529999608, 0.2860378993814758, 0.35914404250153525, 0.3646277907650074 ]"
6,"[ 0.13258485970646555, 0.16951728221183604, 0.18706918176775475, 0.24531838529999608, 0.2860378993814758, 0.35914404250153525, 0.3646277907650074 ]"
7,"[ 0.16951728221183604, 0.18706918176775475, 0.1906571755434092, 0.24531838529999608, 0.2860378993814758, 0.3015881775924716, 0.3646277907650074 ]"
8,"[ 0.16951728221183604, 0.18706918176775475, 0.1906571755434092, 0.24531838529999608, 0.2860378993814758, 0.3015881775924716, 0.3646277907650074 ]"
9,"[ 0.16951728221183604, 0.18706918176775475, 0.1906571755434092, 0.2529861761137882, 0.2860378993814758, 0.3015881775924716, 0.3568398623293629 ]"
